# Multi-Armed Bandit 

Multi-Armed Bandits are a type of Reinforcement Learning algorithmn that is used to learn the best actions to maximize an expected gain. The algorithm learns about the distribution of the gain for the different actions over time by using an exploration(aquire new knowledge and exploitation(optimize the decisions based on existing knowledge) tradeoff stragegy. The agent attempts to balance these competing tasks in order to maximize the total gain over time.


## Epsilon Greedy
$\epsilon$ = probability of exploration.  This is the is the percentage of exploration actions the agent will take. For example if $\epsilon$=0.05, then 5% of actions will be exploration, and 95% will be exploitation. 

Epsilon Greedy allows the agent to explore all actions. The agent learns the best actions by updating the mean of each bandit after each action is taken. 

After the optimal actions are identified, the epsilon greedy algorithm will still explore suboptimal actions at the rate of $\epsilon$.

## Optimistic Initialization
Optimistic intialization is an alternative to the epsilon greedy algorithm. It forces the agent to explore all actions by setting the inital mean of all the bandits as very high, ie an upper limit. Means should be selected to be much higher than what the true mean could be. This forces the agent to explore all the bandits, and the bandit's means will decrease quickly. 

## UCB1
Similar to optimistic initalization, UCB1 initializes the mean of the bandits using an upper bound. This upper bound is based on confidence bounds and is determined by the Hoeffding’s Inequality.
$$P\{|\bar{X}-\mu| \geq \epsilon\} \leq 2e^{-2\epsilon^2N}$$
which rearranges to an upper bound on the mean for the jth bandit
$$X_{UCB-j}=\bar{X_j}+\sqrt{2\frac{lnN}{N_j}}$$
Where N is the number of times all bandits have been played. 

This allows the initialized mean values of the bandits to shrink as the agent tries each bandit, and becomes more confident in our estimate of the bandits true mean. 

## Bayesian Multi-Armed Bandits

In the bayesian multiarmed bandit, the an upper limit is used to initalize the means of the bandits. The upper limit is a upper confidence bound calculated using a distribution of the mean given the observed data, $p(\mu|X)$. This is the posterior of $\mu$.

We use bayes rule to calculate the posterior. 

$$p(\mu|X)=p(X|\mu)p(\mu)$$

We assume X follow a normal $X \sim Normal(\mu, \sigma^2/N)$.

And we put a prior on the mean $\mu \sim Beta(a, b)$

The upper limit is selected as the max of the samples from each of the bandits. The distribution is fat )has large confidence intervals) when few samples have been observed, and becomes skinnier as we approximate the true mean. 

In [1]:
from src.reinforcement_learning import GreedyBandit, OptimisticBandit, UCB1Bandit, BayesianBandit

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed
[nltk_data]     (_ssl.c:852)>


In [2]:
bandits = GreedyBandit.perform_epsilon_greedy(bandit_true_means=[0.01, 0.05, 0.5], N=1000, epsilon=0.1)
for b in bandits: print(b.mean)   

-0.2943410919080815
0.12452945435739586
0.5058750709062644


In [3]:
bandits = OptimisticBandit.perform_optimistic_initialization(bandit_true_means=[0.01, 0.05, 0.5], N=1000, upper_limit=10)
for b in bandits: print(b.mean)

0.01
0.05
10.01422818586346


In [4]:
bandits = UCB1Bandit.perform_ucb1(bandit_true_means=[0.01, 0.05, 0.5], N=1000, upper_limit=10)
for b in bandits: print(b.mean)

0.010878899532350206
-0.3471834886352026
0.4943624580534103


In [5]:
bandits = BayesianBandit.perform_bayesian_bandits(bandit_true_means=[0.01, 0.05, 0.5], N=1000)
for b in bandits: print(b.mean)

-0.0008822362175584519
-0.0025015484072800163
0.5244307547234343
